In [1]:
data_file = "../data/drugs_29-Mar-2021-19_18_52_ATC_N02.txt"

In [2]:
fo = open(data_file,'r', encoding="ISO-8859-1")
header = fo.readline()
header = header.strip().split("\t")

In [3]:
header

['Drug ID',
 'Drug Name',
 'CAS Number',
 'Chemical Name',
 'New Molecular Entity',
 'Drug Interactions',
 'Comparative Efficacy',
 'Highest Development Phase',
 'Indication',
 'Molecular Formula',
 'Mechanism Of Action',
 'Organisation',
 'Pharmacodynamic Characteristics',
 'Route Of Administration',
 'Route Of Elimination',
 'Pharmacokinetic Characteristics',
 'Adverse Events',
 'Therapeutic Class (EphMRA)',
 'Therapeutic Class (WHO)',
 'Synonyms/Brand Names',
 'Antimicrobial Activity',
 'Immunogenicity',
 'Last Significant Update Date',
 'Therapeutic Area',
 'Orphan Designation',
 'License Availability',
 'Brands',
 'On Fastrack',
 'Fastrack country',
 'On PRIME',
 'PRIME country',
 'Biomarkers-Inclusion',
 'Biomarkers-Exclusion',
 'Biomarkers-Outcome Measure',
 'Biomarkers-Safety',
 'Biomarkers-not specified',
 'URL']

In [4]:
ind = 8; moa = 10

In [5]:
therapeutic = [header.index(x) for x in header if 'Therapeutic' in x]
print(therapeutic)

[17, 18, 23]


In [6]:
indicators = []
mechanisms = []
thEs = []
thWs = []
thAs = []

for line in fo:
    line = line.strip().split('\t')
    rowid = line[0]
    try:
        indicator = line[ind]
        mechanism = line[moa]
        thE = line[17]
        thW = line[18]
        thA = line[23]
    indicators.append(indicator.split(","))
    mechanisms.append(mechanism.split(","))
    thEs.append(thE.split(","))
    thWs.append(thW.split(","))
    thAs.append(thA.split(","))



SyntaxError: invalid syntax (<ipython-input-6-95a77aa78294>, line 16)

In [ ]:
indicators_flat = [item.strip('\" ').replace('\'','') for sublist in indicators for item in sublist]
indicators_flat = set(indicators_flat)

In [ ]:
mechanisms_flat = [item.strip('\" ').replace('\'','') for sublist in mechanisms for item in sublist]
mechanisms_flat = set(mechanisms_flat)

In [ ]:
thE_flat = [item.strip('\" ').replace('\'','') for sublist in thEs for item in sublist]
thE_flat = set(thE_flat)
thW_flat = [item.strip('\" ').replace('\'','') for sublist in thWs for item in sublist]
thW_flat = set(thW_flat)
thA_flat = [item.strip('\" ').replace('\'','') for sublist in thAs for item in sublist]
thA_flat = set(thA_flat)

In [ ]:
len(mechanisms_flat)

In [ ]:
len(indicators_flat)

In [ ]:
print(len(thE_flat))
print(len(thW_flat))
print(len(thA_flat))

In [ ]:
import pandas as pd

In [ ]:
df = pd.read_table('../data/drugs_29-Mar-2021-19_18_52_ATC_N02.txt', encoding="ISO-8859-1")

In [ ]:
df.fillna('', inplace=True)

In [7]:
df['Therapeutic Class (EphMRA)']

NameError: name 'df' is not defined

In [ ]:
cleaned_indication = df.set_index('Drug ID').Indication.str.split(',', expand=True).stack().reset_index(level=1, drop=True).to_frame('Indication')

In [ ]:
cleaned_indication.Indication = cleaned_indication.Indication.str.strip(" ")

In [ ]:
pd.set_option("min_rows", 20)
cleaned_indication.Indication.value_counts().head(20)

In [ ]:
ind = pd.get_dummies(cleaned_indication, prefix='IND').groupby(level=0).sum()

In [ ]:
cleaned_mechanism = df.set_index('Drug ID')['Mechanism Of Action'].str.split(',', expand=True).stack().reset_index(level=1, drop=True).to_frame('Mechanism')
cleaned_mechanism.Mechanism = cleaned_mechanism.Mechanism.str.strip(" ")
mch = pd.get_dummies(cleaned_mechanism, prefix='MCH').groupby(level=0).sum()

In [ ]:
cleaned_therape = df.set_index('Drug ID')['Therapeutic Class (EphMRA)'].str.split(',', expand=True).stack().reset_index(level=1, drop=True).to_frame('TherapE')
cleaned_therape.TherapE = cleaned_therape.TherapE.str.strip(" ")
the = pd.get_dummies(cleaned_therape, prefix='THE').groupby(level=0).sum()

In [ ]:
cleaned_therape.TherapE.value_counts()

In [ ]:
cleaned_therapw = df.set_index('Drug ID')['Therapeutic Class (WHO)'].str.split(',', expand=True).stack().reset_index(level=1, drop=True).to_frame('TherapW')
cleaned_therapw.TherapW = cleaned_therapw.TherapW.str.strip(" ")
thw = pd.get_dummies(cleaned_therapw, prefix='THW').groupby(level=0).sum()

In [ ]:
cleaned_therapw.TherapW.value_counts()

In [ ]:
cleaned_therapa = df.set_index('Drug ID')['Therapeutic Area'].str.split(',', expand=True).stack().reset_index(level=1, drop=True).to_frame('TherapA')
cleaned_therapa.TherapA = cleaned_therapa.TherapA.str.strip(" ")
tha = pd.get_dummies(cleaned_therapa, prefix='THA').groupby(level=0).sum()

In [ ]:
cleaned_therapa.TherapA.value_counts()

In [ ]:
df = df.set_index('Drug ID').join(ind).join(mch).join(the).join(thw).join(tha)

In [ ]:
df.to_csv('../data/drugs.txt',sep="\t")

In [ ]:
ind.shape

In [ ]:
mch.shape

# Market-Basket Analysis

In [ ]:
#!conda install MLxtend --channel conda-forge --yes

In [ ]:
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules

In [ ]:
frequent_indicators = apriori(ind, min_support=0.01, use_colnames=True, max_len=4)

In [ ]:
frequent_indicators.sort_values(by=['support'],ascending=False)

In [ ]:
rules = association_rules(frequent_indicators, metric="lift", min_threshold=1)


In [ ]:
rules[ (rules['lift'] >= 2) &
       (rules['confidence'] >= 0.1) ].sort_values(by=['confidence'],ascending=False)

In [ ]:
frequent_mechanisms = apriori(mch, min_support=0.01, use_colnames=True, max_len=4)
frequent_mechanisms.sort_values(by=['support'],ascending=False)

In [ ]:
rules_mch = association_rules(frequent_mechanisms, metric="lift", min_threshold=1)
rules_mch[ (rules_mch['lift'] >= 2) &
       (rules_mch['confidence'] >= 0.1) ].sort_values(by=['confidence'],ascending=False)

In [ ]:
rules_mch['lhs items'] = [len(x) for x in rules_mch['antecedents']]
rules_mch[rules_mch['lhs items']>1].sort_values('lift', ascending=False).head()

In [ ]:
df['Drug Name']

In [ ]:
frequent_all = apriori(mch.join(ind).join(tha), min_support=0.01, use_colnames=True, max_len=6)
frequent_all.sort_values(by=['support'],ascending=False)

In [ ]:
rules_all = association_rules(frequent_all, metric="lift", min_threshold=1)
rules_all[ (rules_all['lift'] >= 2) &
       (rules_all['confidence'] >= 0.4) ].sort_values(by=['lift'],ascending=False)

In [ ]:
df['Adverse Events'].value_counts()

In [ ]:
df[df['Adverse Events'].str.contains("Aggression")]